# createAccount: 
## DONE

In [41]:
import datetime
import hashlib
from random import uniform

import mysql.connector

# Digital Signature
import nacl.encoding
import nacl.signing


Connection = mysql.connector.connect(host='localhost', username='root', password='123456', database='Blockchain')

if(Connection.is_connected()):
	
	cursor = Connection.cursor()

	Name = input("Enter Name: ")
	Age = input("Enter Age: ")
	DOB = input("Enter Date of Birth: \nDate: ")
	MOB = input("Month: ")
	YOB = input("Year: ")


	today = datetime.date.today()
	import base64

	s = Name+DOB+MOB+YOB+str(today.day)+str(today.month)+str(today.year) + str(uniform(0, 100000000000000))

	seed = s.encode('utf-32')
	seed = seed[:32]


	Private_KEY = nacl.signing.SigningKey(seed=seed).generate()
	Public_KEY = Private_KEY.verify_key

	Private_KEY_hex = Private_KEY.encode(encoder=nacl.encoding.HexEncoder).decode()
	Public_KEY_hex = Public_KEY.encode(encoder=nacl.encoding.HexEncoder).decode()

	hashedPrivateKey = str(hashlib.sha256((Private_KEY_hex).encode()).hexdigest())

	query = '''INSERT INTO Users VALUES ('{Public_KEY}', '{Hashed_Private_Key}', 0, 0, 0)'''.format(Public_KEY = Public_KEY_hex, Hashed_Private_Key = hashedPrivateKey)

	cursor.execute(query)

	Connection.commit()
	cursor.close()
	Connection.close()

	print("This is Your Public_KEY:", Public_KEY_hex)
	print("This is your Private_KEY(Do not Share):", Private_KEY_hex)
else:
	print("There was some error, please try again.")

This is Your Public_KEY: 606945fdcedbc1dafe6f9dbd0336601df2b1065d7f328652a47e49555ea989cb
This is your Private_KEY(Do not Share): 5d66af2be6fc5a608782985598726034962d849b058aabbfcf0fd03d41df9182


# Dashboard: 
## DONE

In [49]:
import hashlib
import DashBoard_Commands

import mysql.connector

Connection = mysql.connector.connect(host='localhost', username='root', password='123456', database='Blockchain')

if(Connection.is_connected()):
	
	cursor = Connection.cursor()
	# Logging in Process
	pkey_ = input("Enter Public Key: ")
	Hash_priv_ = str(hashlib.sha256(input("Enter Private Key: ").encode()).hexdigest())

	query = '''SELECT Hashed_Private_Key FROM Users WHERE Public_Key="{Public_Key}"'''.format(Public_Key = pkey_)

	cursor.execute(query)

	try:
		Hashed_Private_Key = cursor.fetchall()[0][0]
	except:
		print("Invalid Public Key")

	cursor.close()
	Connection.close()

	if(Hashed_Private_Key == Hash_priv_):
		print("Login Successful")
	else:
		print("Login Unsuccessful, Incorrect Public/Private Key")
		exit()

	action = ' '

	# Dashboard Commmands
	while(action not in "Ee"):
		print("Enter T to issue a Transaction")
		print("Enter B to Check Balance")
		print("Enter S to Submit Nonce for Ongoing Block")
		print("Enter E to Exit and Log out")
		action = input()	
		if(action not in "TtBbSsEe"):
			print("Invalid Action\n\n")
		elif(action in "Tt"):
			Reciever_ID = input("Enter reciever's ID: ")
			amount = float(input("Enter Transaction Amount: "))

			DashBoard_Commands.transaction(pkey_, Reciever_ID, amount)

			print('\n')
			#Transaction Code Goes Here
		elif(action in "Bb"):
			DashBoard_Commands.balance(pkey_)

			print('\n\n')
			#Balance Check Code Goes Here
		elif(action in "Ss"):
			nonce = input("Enter the Nonce for current block: ")
			DashBoard_Commands.submit_nonce(nonce)
			#Submiting Nonce Code Goes Here

	print("Exited, and Logged out")
else:
	print("There was some error, please try again.")

Login Successful
Enter T to issue a Transaction
Enter B to Check Balance
Enter S to Submit Nonce for Ongoing Block
Enter E to Exit and Log out


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\Sunil\\Desktop\\Abhijeet\\TSS\\CS\\Grade 12 Project\\BlockChain\\Accounts.csv'

# Dashboard Commands:
## NOT DONE

### Balance - DONE
### Transactions - NOT DONE
### Submit Nonce - NOT DONE

In [47]:
import pandas as pd
import nacl.signing
import nacl.encoding

import mysql.connector



def transaction(sender_id, receiver_id, amount, signing_key):
	if(amount < 0): 
		print("Invalid Amount")
		return
	# Acc = pd.read_csv(r'C:\Users\Sunil\Desktop\Abhijeet\TSS\CS\Grade 12 Project\BlockChain\Accounts.csv')
	Connection = mysql.connector.connect(host='localhost', username='root', password='123456', database='Blockchain')
	cursor = Connection.cursor()
	query = '''SELECT Balance FROM Users WHERE Public_Key="{Public_Key}";"'''.format(Public_Key = sender_id)
	cursor.execute(query)
	sender_bal = cursor.fetchall()
	query = '''SELECT Balance FROM Users WHERE Public_Key="{Public_Key}";"'''.format(Public_Key = receiver_id)
	cursor.execute(query)

	receiver_bal = cursor.fetchall()
	if(len(sender_bal) == 0):
		print("Invalid Sender's ID")
		return
	if(len(receiver_bal) == 0):
		print("Invalid Reciever's ID")
		return
	if(sender_bal < amount):
		print("Insufficient Balance")
		return
	
	receiver_bal = receiver_bal[0][0]
	sender_bal = sender_bal[0][0]

	# Transaction in form of json to store in sql
	transaction = '''{
		"sender_id": "{sender_id}",
		"receiver_id": "{receiver_id}",
		"amount": "{amount}"
	}'''.format(sender_id=sender_id, receiver_id=receiver_id, amount=amount)

	# Regenerating key from hex
	sign_key = nacl.signing.SigningKey(signing_key, encoder=nacl.encoding.HexEncoder)

	signed_transaction = sign_key.sign(transaction)

	
    

def balance(user_id):
	Connection = mysql.connector.connect(host='localhost', username='root', password='123456', database='Blockchain')
	cursor = Connection.cursor()
	query = '''SELECT Balance FROM Users WHERE Public_Key="{Public_Key}";"'''.format(Public_Key = user_id)

	cursor.execute(query)

	bal = cursor.fetchall()[0][0]

	print("Balance:", bal)

def submit_nonce(nonce):
	pass

In [38]:
transaction('a367f27015aef274485911c9eba41c06d9d60969ecb68f4083579ea6ae2773da', '78759c7ca24bbdc41bfed933f9c57994812813e35f946b0e531736bda24eca4b', 5.0, '8d2e7996450d389f32246dcb3d9e495c439e705738aa566609da50d5dd9f0104')

DatabaseError: 2014 (HY000): Commands out of sync; you can't run this command now